# Exercise 13 

## CIFAR-10 classification using Keras Conv2D

The CIFAR-10 small photo classification problem is a standard dataset used in computer vision and deep learning.



## CIFAR-10 Photo Classification Dataset

CIFAR comes from the Canadian Institute For Advanced Research and the CIFAR-10 dataset was developed along with the CIFAR-100 dataset by researchers at the CIFAR institute. The dataset is comprised of 60,000 32×32 pixel color photographs of objects from 10 classes with following class labels:
0. airplane
1. automobile
2. bird
3. cat
4. deer
5. dog
6. frog
7. horse
8. ship
9. truck

It is relatively straightforward to achieve 80% classification accuracy. Top performance on the problem is achieved by deep learning convolutional neural networks with a classification accuracy above 90% on the test dataset.

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD


In [ ]:
# test harness for evaluating models on the cifar10 dataset


# load train and test dataset
def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm

# define cnn model
def define_model():
    img_rows, img_cols, channels = 32,32,3
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', input_shape = (img_rows, img_cols, channels), padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128, activation = 'relu', kernel_initializer = 'he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
  
    # compile model
    opt = SGD(learning_rate = 0.001, momentum=0.9)
    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
	  # prepare iterator
    it_train = datagen.flow(trainX, trainY, batch_size=64)
    # fit model 
    steps = int(trainX.shape[0] / 64)
    history = model.fit(it_train, steps_per_epoch = steps, epochs=400, validation_data=(testX, testY), verbose=1)
    #history = model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY), verbose=1) 
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

Epoch 1/400
781/781 [==============================] - 32s 37ms/step - loss: 2.1346 - accuracy: 0.2942 - val_loss: 1.5455 - val_accuracy: 0.4401
Epoch 2/400
781/781 [==============================] - 29s 37ms/step - loss: 1.6204 - accuracy: 0.4075 - val_loss: 1.5100 - val_accuracy: 0.4503
Epoch 3/400
781/781 [==============================] - 29s 38ms/step - loss: 1.5040 - accuracy: 0.4492 - val_loss: 1.5536 - val_accuracy: 0.4404
Epoch 4/400
781/781 [==============================] - 28s 36ms/step - loss: 1.4278 - accuracy: 0.4783 - val_loss: 1.5925 - val_accuracy: 0.4326
Epoch 5/400
781/781 [==============================] - 28s 36ms/step - loss: 1.3567 - accuracy: 0.5057 - val_loss: 1.4686 - val_accuracy: 0.4726
Epoch 6/400
781/781 [==============================] - 28s 36ms/step - loss: 1.3071 - accuracy: 0.5278 - val_loss: 1.3604 - val_accuracy: 0.5123
Epoch 7/400
781/781 [==============================] - 28s 36ms/step - loss: 1.2629 - accuracy: 0.5417 - val_loss: 1.3739 - val_ac

# Conclusion

I started by following the hinted material (https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/ ) and that got me to 74.42 accuracy on validation data. The model overfitted around 60 epochs: training accuracy reached 1.000 and validation accuracy got stuck around 70%.

I checked out several practical examples of dropout layers. Many of the latter recommended adding them before Dense layers and sticking close to 0.5. Decided to add only one with 0.4 before the only Dense layer I had at this point. This got me over 80% val_accuracy with steady couple percentage points higher training accuracy. The training time was still very fast with Google Colab GPU.

Next I followed the hinted material again and added dropout layers. Quite similar results. 

Adding data augmentation on top of that made the training lot slower (approx. 2 hours). Val_accuracy stayed above 80% after 81 epochs. It kept getting better on avarage, though slowly and ended at 85,35% at 200 epochs. Training accuracy was 87,56%. 

I added batch normalization, following again the example. My last run ended after 2h 23 min at 252 epochs with loss: 0.3584 - accuracy: 0.8756 - val_loss: 0.4079 - val_accuracy: 0.8693. The model stayed over 80% on val_accuracy after 75 epochs (train_acc 79,61%) and kept very slowly rising from there. The val_accuracy had stayed above 86% after 194 epochs. Colab won't currently offer me more GPU time, so I will hand this in.

Even if it took more time, I enjoyed doing this in steps, as it allowed me to get better picture of the effects. With the limited resources anything heavier or trial or learn is hard unless you are able to get good results quickly. I found an interesting tutorial, but that would require even more resources: https://medium.com/fenwicks/tutorial-2-94-accuracy-on-cifar10-in-2-minutes-7b5aaecd9cdd